In [22]:
import numpy as np
import pandas as pd
import redis

In [23]:
#configure redis database
hostname = 'redis-15138.c16.us-east-1-2.ec2.cloud.redislabs.com'
portnumber = 15138
password = 'O4FvRrK47ctKCN73mpGfTXOOQVT7Uw1h'

r = redis.StrictRedis(host=hostname,
                  port=portnumber,
                  password=password)

In [24]:
r.ping()

True

In [25]:
#Load the data (numpy zip)
np_file = np.load('dataframe_students_teachers.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values, columns=col_names)
df

,name,role,facial_features
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ..."
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613..."
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999..."
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597..."
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732..."


In [26]:
df['name_role'] = df['name']+'@'+df['role']
df

,name,role,facial_features,name_role
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Scarlett Johansson@Student


In [27]:
records = df[['name_role','facial_features']].to_dict(orient='records')
len(records)

5

In [28]:
###Save in redis

In [29]:
for record in records:
    name_role = record['name_role']
    vector = record['facial_features']
   
    #convert numpy array into bites
    vector_bytes = vector.tobytes()
    #save data into redis cloud
    r.hset(name='company:register',key=name_role,value=vector_bytes)

In [30]:
###Registration form

In [31]:
import cv2
from insightface.app import FaceAnalysis

In [32]:
#configuring insightface
faceapp = FaceAnalysis(name='buffalo_sc',
                      root='insightface_model',
                      providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0, det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
person_name = input('Enter your name ')
trials = 3
for i in range(trials):    
    role = input("""
    Please choose either 1 or 2:
    1. Worker
    2. Admin
    """)
    if role in ('1', '2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid entry, try again')
    if i == 2:
        print('Trials limit exceeded')

key = person_name +'@'+ role
print('your name: ',person_name)
print('your role: ',role)
print('key: ',key)

In [ ]:
#collecting samples

In [ ]:
cap = cv2.VideoCapture(0)
face_embeddings = []
sample = 0
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # if frame is read correctly ret is True
    if ret == False:
        print("Unable to read camera")
        break
    #Get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0,1))
        
        #Extracting facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
    
    sample += 1 
    if sample >= 200:
        break
        
    # Display the resulting frame
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
len(face_embeddings)

In [ ]:
### Optimize data and save it in the database

In [ ]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

In [ ]:
#convert into bytes
x_mean_bytes = x_mean.tobytes()

In [ ]:
#save key and facial bytes into redis database
r.hset(name='company:register',key=key,value=vector_bytes)